In [30]:
import pandas as pd
import numpy as np
import urllib2
import re
import requests
import json
from BeautifulSoup import BeautifulSoup

In [3]:
podcastDf = pd.read_pickle('pkl/english_podcasts.pkl')

In [41]:
# small sample for testing
podcastTest = podcastDf.sample(n = 20)

In [12]:
itunesUrl = 'https://itunes.apple.com/search?'
parameters = {}
parameters['entity'] = 'podcast'
parameters['attribute'] = 'titleTerm'

requestString = ''
for param in parameters:
    paramString = param + '=' + parameters[param]
    if len(requestString) == 0:
        requestString = paramString
    else:
        requestString = requestString + '&' + paramString

In [25]:
def convert_unicode(input):
    if isinstance(input, dict):
        return {convert_unicode(key): convert_unicode(value) for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [convert_unicode(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

In [ ]:
itunes_collection_id = []
mean_rating = []
num_ratings = []

scrapeStem = 'https://itunes.apple.com/us/podcast/id'

for ind, row in podcastDf.iterrows():
    # itunes search request
    requestUrl = itunesUrl + requestString + '&term=' + urllib2.quote(row['name'])
    r = requests.get(requestUrl)
    data = r.json()
    data = data['results']
    
    # skip if no matches or >1 match
    if len(data) != 1:
        itunes_collection_id.append(np.nan)
        mean_rating.append(np.nan)
        num_ratings.append(np.nan)
        continue
    
    # scrape itunes page for ratings
    data = data[0]
    data = convert_unicode(data)
    itunes_collection_id.append(data['collectionId'])
    scrapeUrl = scrapeStem + str(data['collectionId'])
    soup = BeautifulSoup(urllib2.urlopen(scrapeUrl).read())
    thisMean = convert_unicode(re.findall(r'Customer Ratings(\d\.\d+)', soup.text))
    thisTotal = convert_unicode(re.findall(r'&nbsp;(\d+) Ratings', soup.text))
    mean_rating.append(thisMean)
    num_ratings.append(thisTotal)

started above at 4:59 pm on Weds